In [125]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [126]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, fitGMM

from auctions.webutils import *

from scipy.stats import loguniform
import copy

import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Magma256
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


from open_spiel.python.examples.ubc_cma import analyze_checkpoint

output_notebook()


Loading BokehJS ...

In [127]:
# load experiment
experiment = Experiment.objects.get(name='new_configs_dec_6')

In [128]:
# For each run, I want to know: ApproxNashConv, Auction Length, Revenue, Potential Function, Game Name, Game Mod

In [139]:
records = []
for run in experiment.equilibriumsolverrun_set.all():
    record = dict(game_name=run.game.name, potential=run.config.get('potential_function', 'None'), run_name=run.name)
    
    # TODO: Should I consider ALL checkpoints and do a frontier tradeoff? Or just the last?
    
    # Get the last eval
    final_checkpoint = run.equilibriumsolverruncheckpoint_set.last()
    record = {**record, **analyze_checkpoint(final_checkpoint)}
    record['t'] = final_checkpoint.t
    
    # Get NC information -> requires best responses
    ev_df = parse_run(run)
    nc = ev_df.query(f't == {final_checkpoint.t}')['ApproxNashConv'].max()
    record['NashConv'] = nc
    
    records.append(record)

RelatedObjectDoesNotExist: EquilibriumSolverRunCheckpoint has no evaluation.

In [ ]:
df = pd.DataFrame.from_records(records)

In [ ]:
# TODO: NashConv varies for min and max - you want the CORRESPONDING one to be visible in a column next to it? 
describe_df = df.groupby(['game_name', 'potential'])[['auction_lengths', 'total_revenue']].describe()
cols = [c for c in describe_df.columns if 'min' in c[1] or 'max' in c[1]]
describe_df = describe_df[cols]
describe_df

KeyError: 'game_name'

In [53]:
df.query('game_name.str.contains("regional_vs_national")').groupby('game_name')['auction_lengths'].describe()

,count,mean,std,min,25%,50%,75%,max
game_name,,,,,,,,
regional_vs_national.json,18.0,1.771605,0.950654,1.0011,1.001626,1.005202,2.957947,3.242549
regional_vs_national_signal.json,18.0,2.023574,0.774502,1.0000,1.273418,1.989287,2.778067,3.047424


In [71]:
df.query('run_name == "regional_vs_national-nov23ppo_3_auction_length-102"')#['common_allocations'].item()

,game_name,potential,run_name,p0_utility,p0_payment,p1_utility,p1_payment,total_welfare,total_revenue,auction_lengths,common_allocations,t,NashConv
49,regional_vs_national.json,auction_length,regional_vs_national-nov23ppo_3_auction_length...,1549.857086,1235.516013,1661.923239,1554.500801,6001.797138,2790.016813,2.0001,"[((1, 1, 2), 7479), ((1, 2, 1), 2513)]",9999872,0.744356


In [70]:
pd.Series(output['raw_rewards'][0]).mean()

1550.1021114780347

In [63]:
EquilibriumSolverRun.objects.get(name='regional_vs_national-nov23ppo_3_auction_length-102').equilibriumsolverruncheckpoint_set.last().bestresponse_set.last()

<BestResponse: Player 0 BR (nov23br/ppo_3) to regional_vs_national-nov23ppo_3_auction_length-102 (new_configs_nov_29) Iteration 9999872>

In [ ]:
from auctions.management.commands.ppo_eval import eval_command

### Note: because of slurm queue timing, I can't guarantee anything about WHEN the BRs will exist from past iterations. How might one eventualy run this? ###

### RUN AN OLD BEST RESPONSE ### 


### STEP 1: Get all of the old best responses ###


best_response = BestResponse.objects.get(
    checkpoint = equilibrium_solver_run_checkpoint,
    br_player = br_player,
    name = br_name
)
br_agent = load_ppo_agent(best_response)
env_and_policy.agents[br_player] = br_agent 

# RUN EVAL
eval_output = run_eval(env_and_policy, num_samples=num_samples, report_freq=report_freq, seed=seed, compute_efficiency=compute_efficiency)


output = eval_command(9999872, "new_configs_nov_29", "regional_vs_national-nov23ppo_3_auction_length-102", 'nov23br/ppo_3', 0, True, 1234, 1000, 10_000, False)

In [72]:
run = EquilibriumSolverRun.objects.get(name='regional_vs_national-nov23ppo_3_auction_length-102')

In [113]:
records = []
relevant_brs = []
last_checkpoint = run.equilibriumsolverruncheckpoint_set.last()

# Step 1: Get all the BRs
for checkpoint in tqdm(run.equilibriumsolverruncheckpoint_set.all()):
    if checkpoint == last_checkpoint:
        continue
    for best_response in checkpoint.bestresponse_set.filter(~Q(name__icontains='straightforward')):
        relevant_brs.append(best_response)
        
# Step 2: For the last checkpoint, run all the BRs
for best_response in relevant_brs:
    env_and_policy = ppo_db_checkpoint_loader(last_checkpoint)
    br_agent = load_ppo_agent(best_response)
    br_player = best_response.br_player

    new_br = BestResponse.objects.create(
            checkpoint = last_checkpoint,
            br_player = br_player,
            walltime = 0,
            model = best_response.model,
            config = best_response.config,
            name = best_response.name + f'_for_t={best_response.checkpoint.t}',
            t = best_response.t
        )

    env_and_policy.agents[br_player] = br_agent 
    eval_command(last_checkpoint.t, run.experiment.name, run.name, new_br.name, br_player, False, 1234, 1000, 10_000, False)        
            


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 395.98it/s]


In [114]:
df = parse_run(run)

In [115]:
# BestResponse.objects.filter(checkpoint__equilibrium_solver_run=run, name__contains='for_t=').delete()

In [116]:
df.query('t == 9999872')

,t,name,reward,br_player,player,median_length,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model,game
18,9999872,straightforward,1069.173010,0,0,NaN,1549.857086,-480.684075,0.000000,60.773564,61.272895,2,regional_vs_national-nov23ppo_3_auction_length...,regional_vs_national.json
19,9999872,nov23br/ppo_3,1550.102111,0,0,NaN,1549.857086,0.245026,0.245026,60.773564,61.272895,2,regional_vs_national-nov23ppo_3_auction_length...,regional_vs_national.json
20,9999872,nov23br/ppo_3_for_t=2500096,1518.517664,0,0,NaN,1549.857086,-31.339422,0.000000,60.773564,61.272895,2,regional_vs_national-nov23ppo_3_auction_length...,regional_vs_national.json
21,9999872,nov23br/ppo_3_for_t=5000192,1607.700793,0,0,NaN,1549.857086,57.843708,57.843708,60.773564,61.272895,2,regional_vs_national-nov23ppo_3_auction_length...,regional_vs_national.json
22,9999872,nov23br/ppo_3_for_t=7500032,1610.630650,0,0,NaN,1549.857086,60.773564,60.773564,60.773564,61.272895,2,regional_vs_national-nov23ppo_3_auction_length...,regional_vs_national.json
23,9999872,None,1549.857086,None,0,2.0,1549.857086,0.000000,0.000000,60.773564,61.272895,2,regional_vs_national-nov23ppo_3_auction_length...,regional_vs_national.json
24,9999872,straightforward,1288.056669,1,1,NaN,1661.923239,-373.866569,0.000000,0.499330,61.272895,2,regional_vs_national-nov23ppo_3_auction_length...,regional_vs_national.json
25,9999872,nov23br/ppo_3,1662.422569,1,1,NaN,1661.923239,0.499330,0.499330,0.499330,61.272895,2,regional_vs_national-nov23ppo_3_auction_length...,regional_vs_national.json
26,9999872,nov23br/ppo_3_for_t=2500096,1655.688876,1,1,NaN,1661.923239,-6.234363,0.000000,0.499330,61.272895,2,regional_vs_national-nov23ppo_3_auction_length...,regional_vs_national.json
27,9999872,nov23br/ppo_3_for_t=5000192,1662.342274,1,1,NaN,1661.923239,0.419035,0.419035,0.499330,61.272895,2,regional_vs_national-nov23ppo_3_auction_length...,regional_vs_national.json


In [124]:
f = plot_from_df(df)
show(f)